In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, LSTM, TimeDistributed
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

from tensorflow.keras.optimizers import Adam


In [2]:
parent_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
main_directory = os.path.dirname(parent_directory)
phase_2 =  pd.read_csv(main_directory + '/data/phase_2_data.csv')
phase_2.head()

,ID,From,Date,Hour,Participation_Phase,Demand_kWh,Temperature,Temperature24,Temperature48,Temperature72
0,Exp_3928,2020-12-01 00:00:00+00:00,2020-12-01,1,Phase_2,4.43,4.2,0.7,-0.8,-1.3
1,Exp_3928,2020-12-01 01:00:00+00:00,2020-12-01,2,Phase_2,4.50,4.2,1.0,-0.7,-1.2
2,Exp_3928,2020-12-01 02:00:00+00:00,2020-12-01,3,Phase_2,5.44,3.9,1.3,-0.6,-1.0
3,Exp_3928,2020-12-01 03:00:00+00:00,2020-12-01,4,Phase_2,4.50,3.1,1.6,-0.5,-0.9
4,Exp_3928,2020-12-01 04:00:00+00:00,2020-12-01,5,Phase_2,4.48,2.4,1.8,-0.4,-0.8


In [3]:
unique_ids = phase_2["ID"].unique()[:100]
phase_2 = phase_2[phase_2["ID"].isin(unique_ids)]

In [4]:
phase_2.set_index('From',inplace=True)

phase_2.head()

,ID,Date,Hour,Participation_Phase,Demand_kWh,Temperature,Temperature24,Temperature48,Temperature72
From,,,,,,,,,
2020-12-01 00:00:00+00:00,Exp_3928,2020-12-01,1,Phase_2,4.43,4.2,0.7,-0.8,-1.3
2020-12-01 01:00:00+00:00,Exp_3928,2020-12-01,2,Phase_2,4.50,4.2,1.0,-0.7,-1.2
2020-12-01 02:00:00+00:00,Exp_3928,2020-12-01,3,Phase_2,5.44,3.9,1.3,-0.6,-1.0
2020-12-01 03:00:00+00:00,Exp_3928,2020-12-01,4,Phase_2,4.50,3.1,1.6,-0.5,-0.9
2020-12-01 04:00:00+00:00,Exp_3928,2020-12-01,5,Phase_2,4.48,2.4,1.8,-0.4,-0.8


75/25

In [5]:
df_features = ['Temperature']
df_target = ["Demand_kWh"]

X= phase_2[df_features].values
y=phase_2[df_target].values

X = X.reshape((X.shape[0], 1, X.shape[1])) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

In [6]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[1])).reshape(X_train.shape)
X_test = scaler_X.transform(X_test.reshape(-1, X_test.shape[1])).reshape(X_test.shape)

y_train = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

In [7]:
model = Sequential()
model.add(LSTM(units=64, activation='relu', input_shape=(1, 1))) 
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer, loss='mse')

c:\Users\alfre\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
history = model.fit(X_train, y_train,epochs=10, batch_size=16, validation_data=(X_test, y_test))


Epoch 1/10
13050/13050 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - loss: 0.0097 - val_loss: 0.0085
Epoch 2/10
13050/13050 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - loss: 0.0094 - val_loss: 0.0098
Epoch 3/10
13050/13050 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - loss: 0.0093 - val_loss: 0.0096
Epoch 4/10
13050/13050 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - loss: 0.0094 - val_loss: 0.0086
Epoch 5/10
13050/13050 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - loss: 0.0093 - val_loss: 0.0098
Epoch 6/10
13050/13050 ━━━━━━━━━━━━━━━━━━━━ 13s 957us/step - loss: 0.0093 - val_loss: 0.0099
Epoch 7/10
13050/13050 ━━━━━━━━━━━━━━━━━━━━ 12s 955us/step - loss: 0.0094 - val_loss: 0.0097
Epoch 8/10
13050/13050 ━━━━━━━━━━━━━━━━━━━━ 12s 955us/step - loss: 0.0093 - val_loss: 0.0097
Epoch 9/10
13050/13050 ━━━━━━━━━━━━━━━━━━━━ 13s 960us/step - loss: 0.0093 - val_loss: 0.0084
Epoch 10/10
13050/13050 ━━━━━━━━━━━━━━━━━━━━ 12s 954us/step - loss: 0.0093 - val_loss: 0.0102


In [9]:
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

forecast = model.predict(X_test)
y_pred = scaler_y.inverse_transform(forecast)

forecast_df = pd.DataFrame(data=y_pred, columns=['forecast_data'], index=phase_2.index[:len(y_pred)])

forecast_df = phase_2.join(forecast_df, how="left")

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

2175/2175 ━━━━━━━━━━━━━━━━━━━━ 2s 631us/step - loss: 0.0110
Test Loss: 0.010150841437280178
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1s 540us/step
MSE: 11.255825693040217
MAE: 3.2935577312694218
RMSE: 3.3549702968938813


In [10]:
df = forecast_df[forecast_df["forecast_data"].notna()]
df = df.drop(columns=["Temperature","Temperature24","Temperature48","Temperature72","Hour","Participation_Phase"])

df.to_csv(parent_directory+"/Neural_Network/predictions/phase_2/cnn_lstm_phase_2_method_2_75_25.csv",index=True)

80/20

In [11]:
df_features = ['Temperature']
df_target = ["Demand_kWh"]

X= phase_2[df_features].values
y=phase_2[df_target].values

X = X.reshape((X.shape[0], 1, X.shape[1])) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [12]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[1])).reshape(X_train.shape)
X_test = scaler_X.transform(X_test.reshape(-1, X_test.shape[1])).reshape(X_test.shape)

y_train = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

In [13]:
model = Sequential()
model.add(LSTM(units=64, activation='relu', input_shape=(1, 1))) 
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer, loss='mse')

c:\Users\alfre\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
history = model.fit(X_train, y_train,epochs=10, batch_size=16, validation_data=(X_test, y_test))


Epoch 1/10
13920/13920 ━━━━━━━━━━━━━━━━━━━━ 14s 935us/step - loss: 0.0096 - val_loss: 0.0084
Epoch 2/10
13920/13920 ━━━━━━━━━━━━━━━━━━━━ 13s 907us/step - loss: 0.0095 - val_loss: 0.0070
Epoch 3/10
13920/13920 ━━━━━━━━━━━━━━━━━━━━ 13s 911us/step - loss: 0.0095 - val_loss: 0.0071
Epoch 4/10
13920/13920 ━━━━━━━━━━━━━━━━━━━━ 13s 909us/step - loss: 0.0095 - val_loss: 0.0085
Epoch 5/10
13920/13920 ━━━━━━━━━━━━━━━━━━━━ 13s 907us/step - loss: 0.0095 - val_loss: 0.0089
Epoch 6/10
13920/13920 ━━━━━━━━━━━━━━━━━━━━ 13s 905us/step - loss: 0.0094 - val_loss: 0.0078
Epoch 7/10
13920/13920 ━━━━━━━━━━━━━━━━━━━━ 13s 931us/step - loss: 0.0094 - val_loss: 0.0083
Epoch 8/10
13920/13920 ━━━━━━━━━━━━━━━━━━━━ 12s 885us/step - loss: 0.0094 - val_loss: 0.0077
Epoch 9/10
13920/13920 ━━━━━━━━━━━━━━━━━━━━ 12s 892us/step - loss: 0.0094 - val_loss: 0.0076
Epoch 10/10
13920/13920 ━━━━━━━━━━━━━━━━━━━━ 13s 902us/step - loss: 0.0095 - val_loss: 0.0076


In [15]:
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

forecast = model.predict(X_test)
y_pred = scaler_y.inverse_transform(forecast)

forecast_df = pd.DataFrame(data=y_pred, columns=['forecast_data'], index=phase_2.index[:len(y_pred)])

forecast_df = phase_2.join(forecast_df, how="left")

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

1740/1740 ━━━━━━━━━━━━━━━━━━━━ 1s 643us/step - loss: 0.0090
Test Loss: 0.007622078061103821
1740/1740 ━━━━━━━━━━━━━━━━━━━━ 1s 564us/step
MSE: 9.195440629700423
MAE: 2.989613750985994
RMSE: 3.032398494541973


In [16]:
df = forecast_df[forecast_df["forecast_data"].notna()]
df = df.drop(columns=["Temperature","Temperature24","Temperature48","Temperature72","Hour","Participation_Phase"])

df.to_csv(parent_directory+"/Neural_Network/predictions/phase_2/cnn_lstm_phase_2_method_2_80_20.csv",index=True)

50/50

In [17]:
df_features = ['Temperature']
df_target = ["Demand_kWh"]

X= phase_2[df_features].values
y=phase_2[df_target].values

X = X.reshape((X.shape[0], 1, X.shape[1])) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=False)

In [18]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[1])).reshape(X_train.shape)
X_test = scaler_X.transform(X_test.reshape(-1, X_test.shape[1])).reshape(X_test.shape)

y_train = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

In [19]:
model = Sequential()
model.add(LSTM(units=64, activation='relu', input_shape=(1, 1))) 
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer, loss='mse')

c:\Users\alfre\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [20]:
history = model.fit(X_train, y_train,epochs=10, batch_size=16, validation_data=(X_test, y_test))


Epoch 1/10
8700/8700 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - loss: 0.0099 - val_loss: 0.0088
Epoch 2/10
8700/8700 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 0.0097 - val_loss: 0.0092
Epoch 3/10
8700/8700 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 0.0095 - val_loss: 0.0102
Epoch 4/10
8700/8700 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 0.0095 - val_loss: 0.0093
Epoch 5/10
8700/8700 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 0.0097 - val_loss: 0.0087
Epoch 6/10
8700/8700 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 0.0096 - val_loss: 0.0103
Epoch 7/10
8700/8700 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - loss: 0.0096 - val_loss: 0.0091
Epoch 8/10
8700/8700 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - loss: 0.0096 - val_loss: 0.0103
Epoch 9/10
8700/8700 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 0.0095 - val_loss: 0.0097
Epoch 10/10
8700/8700 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 0.0097 - val_loss: 0.0105


In [21]:
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

forecast = model.predict(X_test)
y_pred = scaler_y.inverse_transform(forecast)

forecast_df = pd.DataFrame(data=y_pred, columns=['forecast_data'], index=phase_2.index[:len(y_pred)])

forecast_df = phase_2.join(forecast_df, how="left")

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

4350/4350 ━━━━━━━━━━━━━━━━━━━━ 3s 619us/step - loss: 0.0086
Test Loss: 0.010478053241968155
4350/4350 ━━━━━━━━━━━━━━━━━━━━ 3s 563us/step
MSE: 12.416634061039357
MAE: 3.4727756302346475
RMSE: 3.5237244587282017


In [22]:
df = forecast_df[forecast_df["forecast_data"].notna()]
df = df.drop(columns=["Temperature","Temperature24","Temperature48","Temperature72","Hour","Participation_Phase"])

df.to_csv(parent_directory+"/Neural_Network/predictions/phase_2/cnn_lstm_phase_2_method_2_50_50.csv",index=True)